In [1]:
%load_ext lineapy

In [2]:
import xlwings as xw
import json
import datetime
print(datetime.datetime.utcnow())

2022-10-14 18:25:48.831539


# Benchmark Model (Excel VBA)

Here, we are using an existing pricer implemented as an VBA within an Excel file

* Load option parameters
* Load the Excel wookbook
* Demonstrate two ways to utilize the spreadsheet
    + Modify the cell directly and get the result from cell
    + Call the builtin VBA and extract the result directly

Note that, interacting with Excel require you to run the code with access to Excel directly.


## Load option parameters

In [3]:
config_path = '../config/option_config.json'
params = json.load(open(config_path,'r'))

## Open the SpreadSheet

In [4]:
workbook_path = r'BlackSholesGreeksVBA.xlsm'
sheet_name = 'Basic'

In [5]:
wb = xw.Book(workbook_path)
sheet = wb.sheets[sheet_name]

### Manipulate Cells for Input Value and Extract Value from Result Cell and Save the Result as an Artifact

In [6]:
def update_cell_and_get_result(sheet, params):
    option_type = 'C' 
    StockPrice = params['S']
    StrikePrice = params['K']
    RiskFreeRate = params['r']
    Volatility = params['v']
    TimeToExpiry = params['T']
    Dividend = 0    
    sheet.range('StockPrice').value = StockPrice
    sheet.range('StrikePrice').value = StrikePrice
    sheet.range('TimeToExpiry').value = TimeToExpiry
    sheet.range('RiskFreeRate').value = RiskFreeRate
    sheet.range('Volatility').value = Volatility
    sheet.range('Dividend').value = Dividend    

    return sheet.range("E7").value 


In [7]:
excel_cell_price = update_cell_and_get_result(sheet, params)
lineapy.save(excel_cell_price, 'excel_cell_price')

LineaArtifact(name='excel_cell_price', _version=3)

In [8]:
print(lineapy.get('excel_cell_price').get_code())

import json

import xlwings as xw

config_path = "../config/option_config.json"
params = json.load(open(config_path, "r"))
workbook_path = r"BlackSholesGreeksVBA.xlsm"
sheet_name = "Basic"
wb = xw.Book(workbook_path)
sheet = wb.sheets[sheet_name]


def update_cell_and_get_result(sheet, params):
    option_type = "C"
    StockPrice = params["S"]
    StrikePrice = params["K"]
    RiskFreeRate = params["r"]
    Volatility = params["v"]
    TimeToExpiry = params["T"]
    Dividend = 0
    sheet.range("StockPrice").value = StockPrice
    sheet.range("StrikePrice").value = StrikePrice
    sheet.range("TimeToExpiry").value = TimeToExpiry
    sheet.range("RiskFreeRate").value = RiskFreeRate
    sheet.range("Volatility").value = Volatility
    sheet.range("Dividend").value = Dividend

    return sheet.range("E7").value


excel_cell_price = update_cell_and_get_result(sheet, params)



### Direct Use the VBA Function within the Spreadsheet

In [9]:
option_type = 'C' 
StockPrice = params['S']
StrikePrice = params['K']
RiskFreeRate = params['r']
Volatility = params['v']
TimeToExpiry = params['T']
Dividend = 0    

vba_pricing = wb.macro('OptionPrice')(option_type, StockPrice, StrikePrice, TimeToExpiry, RiskFreeRate, Volatility, Dividend)
print(vba_pricing)
lineapy.save(vba_pricing, 'excel_vba_price')

10.403539152996622


LineaArtifact(name='excel_vba_price', _version=2)

In [10]:
print(lineapy.get('excel_vba_price').get_code())

import json

import xlwings as xw

config_path = "../config/option_config.json"
params = json.load(open(config_path, "r"))
workbook_path = r"BlackSholesGreeksVBA.xlsm"
wb = xw.Book(workbook_path)
option_type = "C"
StockPrice = params["S"]
StrikePrice = params["K"]
RiskFreeRate = params["r"]
Volatility = params["v"]
TimeToExpiry = params["T"]
Dividend = 0
vba_pricing = wb.macro("OptionPrice")(
    option_type,
    StockPrice,
    StrikePrice,
    TimeToExpiry,
    RiskFreeRate,
    Volatility,
    Dividend,
)



In [11]:
wb.close()